Optuna uses bayesian optimization to converge to minimum function -> smarter than e.g. gridsearch or randomsearch

### Parameters

In [1]:
data_path = r".\Data\\"
submiss_path = r".\Prediction\\"
print(data_path)
#hallo tijl

.\Data\\


### Imports

In [12]:
# Algemeen
import pandas as pd
import numpy as np
import math

# Model
from sklearn.ensemble import GradientBoostingRegressor

# Hyper-parameter optimizing
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Metric to minimize
from sklearn.metrics import mean_squared_error

# Read Data in DataFrame

In [8]:
# Training
train_set = pd.read_csv(data_path+"train.csv")
train_y = train_set['target']
train_X = train_set.drop(labels = ['target','id'], axis = 1)

# Testing
test_set = pd.read_csv(data_path+"test.csv")
test_X = test_set.drop(labels = 'id', axis = 1)

Optional: remove outliers

In [9]:
def remove_outliers(df):
    try:
        for column_name in df.columns:
            Q1 = df[column_name].quantile([0.25]).values
            Q3 = df[column_name].quantile([0.75]).values
            IQR = Q3-Q1
            df[column_name][(df[column_name]< (Q1-1.5*IQR)[0]) | (df[column_name]> (Q3+1.5*IQR)[0])] = df[column_name].median()
    except: 
        Q1 = df.quantile([0.25]).values
        Q3 = df.quantile([0.75]).values
        IQR = Q3-Q1
        df[(df< (Q1-1.5*IQR)[0]) | (df> (Q3+1.5*IQR)[0])] = df.median()
    return df


train_X = remove_outliers(train_X)
test_X = remove_outliers(test_X)
train_y = remove_outliers(train_y)


<ipython-input-9-5fcb25037763>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[(df< (Q1-1.5*IQR)[0]) | (df> (Q3+1.5*IQR)[0])] = df.median()


# Optuna Hyperparameter Optimization

## Step 1: make objective function

In [26]:
def scorer(estimator, X, y):
    sqrt(return mean_squared_error(estimator.predict(X),y))

In [27]:
def objective_cv(trial, train_X = train_X, train_y = train_y):

    # Define search space
    learning_rate = trial.suggest_loguniform('learning_rate',1e-3,3)
    n_estimators = trial.suggest_int('n_estimators', 2,25)
    max_depth = trial.suggest_int('max_depth', 2,25)

    # make model and perform cross-validation
    model = GradientBoostingRegressor(random_state=34, learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)
    scores = cross_val_score(model, train_X, train_y, cv=5, scoring=scorer)
    return np.mean(scores)

In [37]:
def objective(trial, train_X = train_X, train_y = train_y):
    X_train, X_eval, y_train, y_eval = train_test_split(train_X, train_y, test_size=0.20, random_state=34)
    # Define search space
    learning_rate = trial.suggest_loguniform('learning_rate',1e-3,0.5)
    n_estimators = trial.suggest_int('n_estimators', 5,35)
    max_depth = trial.suggest_int('max_depth', 5,35)

    # make model and perform cross-validation
    model = GradientBoostingRegressor(random_state=34, learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)
    model.fit(X_train,y_train)
    return scorer(model, X_eval, y_eval)

## Step 2: perform study

In [38]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
print('Best trial:', study.best_trial.params)

[I 2022-06-15 01:51:12,216] A new study created in memory with name: no-name-ff3427a4-c4fa-4731-95ce-e6340101db4b
[I 2022-06-15 01:54:46,867] Trial 0 finished with value: 0.5269027329782722 and parameters: {'learning_rate': 0.006961170402421379, 'n_estimators': 10, 'max_depth': 24}. Best is trial 0 with value: 0.5269027329782722.
[I 2022-06-15 02:00:30,193] Trial 1 finished with value: 0.5049902131879822 and parameters: {'learning_rate': 0.053202029678921424, 'n_estimators': 21, 'max_depth': 17}. Best is trial 1 with value: 0.5049902131879822.
[I 2022-06-15 02:02:12,036] Trial 2 finished with value: 0.5155819298488716 and parameters: {'learning_rate': 0.029422154253242593, 'n_estimators': 17, 'max_depth': 8}. Best is trial 1 with value: 0.5049902131879822.
[I 2022-06-15 02:07:10,805] Trial 3 finished with value: 0.509585611397348 and parameters: {'learning_rate': 0.02141285317582601, 'n_estimators': 30, 'max_depth': 18}. Best is trial 1 with value: 0.5049902131879822.
[I 2022-06-15 02:

Best trial: {'learning_rate': 0.053202029678921424, 'n_estimators': 21, 'max_depth': 17}


## Step 3: Visualize learning

In [ ]:
# shows the scores from all trials
optuna.visualization.plot_optimization_history(study)

In [ ]:
# parameter interactions on an interactive chart.
optuna.visualization.plot_contour(study, params=['n_estimators','max_depth'])

In [ ]:
# Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
# Visualize empirical distribution function
optuna.visualization.plot_edf(study)